### 10 самых важных статей по физике высоких энергий в соответствии с
### https://snap.stanford.edu/data/cit-HepTh.html 

In [1]:
import numpy as np

from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix

In [2]:
data = np.loadtxt("cit-HepTh.txt.gz", comments="#", unpack=True)

In [3]:
from collections import Counter
d = 0.85
c = Counter(data[0]) #количество ссылок с каждой статьи на другие

In [4]:
a = set(data[0]) #смотрим есть ли различные элементы в множествах ссылающихся и на тех что ссылаются
b = set(data[1]) #и да, они есть
len(a.difference(b))

4590

In [5]:
all_papers  = np.array(list(a.union(b))) #теперь каждой статье соответсвует индекс в этом массиве

In [6]:
val = [] #подготовим данные для заполнения разреженной матрицы в координатном формате записи
row = []
col = []

for i in range(len(data[0])):
    ind_row, = np.where(all_papers == data[0][i])
    ind_col, = np.where(all_papers == data[1][i])
    row.append(ind_row)
    col.append(ind_col)
    val.append(1./c[data[0][i]]) #кладем значение равное 1/(общее кол-во ссылок с этой страницы), если есть ссылка
    
val = np.array(val)
row = np.reshape(np.array(row), len(row))
col = np.reshape(np.array(col), len(col))

size = len(all_papers) #размер матрицы определяется и также статьями из которых нет ссылок

In [7]:
matrix = coo_matrix((val, (row, col)), shape=(size, size))

A = matrix.tocsr() #матрица в разреженном строчном формате

l = np.array([1 for i in range(size)]) #единичный вектор

In [8]:
#итеративно ищем результат (можно проверить, что на size-1 шаге значения совпадают с значениями на шаге size)
PR = l
for i in range(size):
    PR = A.dot(PR)* d + l * (1-d) 
    PR = PR * size / np.sum(PR)
PR

array([ 0.34650674,  0.43155093,  0.17431701, ...,  0.459899  ,
        0.38902883,  0.45175175])

In [9]:
ind_max = PR.argsort()[-10:][::-1] #индексы максимальных значений PR
max_papers = [all_papers[i] for i in ind_max]
max_papers #вообще вышло 12 максимальных совпадающих значений PR

[9308141.0,
 9301082.0,
 9307086.0,
 9308150.0,
 10273.0,
 6187.0,
 205224.0,
 9404069.0,
 9404072.0,
 9201015.0]